# Career Compass: AI-Powered Career Gap Analyzer with GenAI

## Use Case / Problem Statement:

Many students and professionals—especially those considering a career shift—struggle to understand how to bridge the gap between their current experience and a desired role. Traditional career guidance tools offer broad suggestions without understanding the user's background or intended direction. As a result, users often waste time applying to misaligned roles or studying irrelevant material.

## How GenAI Solves the Problem:

Career Compass empowers users to input their current resume and specify a target role and sector they want to pursue. Generative AI analyzes their current experience to identify transferable skills (with supporting evidence), highlights gaps that need to be filled, and outputs a structured roadmap including learning resources, relevant certifications, and intermediate job titles along the way. This makes career transition more actionable, personalized, and strategic.

## GenAI Capabilities Demonstrated:

* **Document Understanding**: Ingests and parses resumes to extract meaningful experience and skills.
* **Few-Shot Prompting**: Leverages example-driven prompts to improve learning resource recommendations.
* **Structured Output**: Outputs the career plan and search results in a machine-readable format for downstream display or visualization.
* **Grounding**: Uses Google Search to retrieve relevant certification programs, online courses, and other learning content tailored to the gap.

# 1. Install Dependencies

In [ ]:
!pip install -q -U google-genai

# 2. Import Libraries

In [ ]:
import os
import json
import pathlib
import httpx
from pprint import pprint
from google import genai
from google.genai import types
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# 3. Set API Key

In [ ]:
# Code snippet from Kaggle Secrets Manager
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

GEMINI_API_KEY = user_secrets.get_secret("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    raise EnvironmentError("Gemini API key not found. Please set the GEMINI_API_KEY environment variable.")

client = genai.Client(api_key=GEMINI_API_KEY)

# 4. Specify Target Role and Sector 

In [ ]:
# Enter the job title and sector you're targeting

target_role = "Product Manager"
target_sector = "Technology"

# 5. Define Career Plan JSON Schema and Prompt Template

In [ ]:
# Define JSON schema for formatting career plan
career_plan_json_schema = '''
CareerPath = {
    "transferable_skills": list[{
    "name": str,
    "evidence": str
    }],
    "missing_skills": list[str],
    "learning_resources": list[{
      "title": str,
      "provider": str
    }],
    "recommended_certifications": list[{
      "title": str,
      "provider": str
    }],
    "intermediate_roles": list[str],
    "roadmap_summary": str
}
Return: CareerPath as JSON
'''

# Craft a prompt to generate a career plan and format the response according to the career plan schema
career_plan_prompt = f"""
The user wants to transition into a career as a {target_role} in the {target_sector} sector.
Analyze their background and:
- Identify transferable skills they already have, and cite the specific sentence or phrase from their resume or profile that supports each skill
- Identify key skills and certifications they are missing
- Recommend specific learning resources along with the name of the site or company that provides the resource
- Suggest a sequence of intermediate job roles they could pursue in the order of job role hierarchy (lowest to highest), 
and ending with the role they want to transition into.
- Provide a one-paragraph summary of their roadmap, referring to the individual as "you" instead of their name
Format the output using this schema:
{career_plan_json_schema}
"""

# 6. Process Local Resume PDF and Generate Career Plan

In [ ]:
from pathlib import Path

filepath = Path("/kaggle/input/resume/my_resume.pdf")
if not filepath.exists():
    raise FileNotFoundError(" 'my_resume.pdf' was not found. Please upload your resume using the 'Add Data > Upload' menu before running this notebook.")

# Prompt Gemini with the user's resume pdf and the career plan prompt
career_plan_response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[
        types.Part.from_bytes(
            data=filepath.read_bytes(),
            mime_type="application/pdf",
        ),
        career_plan_prompt
    ]
)

# 7. Extract and Parse JSON Career Plan from Gemini Response

In [ ]:
try:
    # Extract the JSON from the career plan response text
    career_plan_response_text = career_plan_response.text
    
    # Look for JSON content within markdown code blocks
    if "```json" in career_plan_response_text:
        # Extract content between ```json and ```
        career_plan_json_text = career_plan_response_text.split("```json")[1].split("```")[0].strip()
    else:
        # If no markdown formatting, use the raw text
        career_plan_json_text = career_plan_response_text
    
    # Now parse the extracted JSON content
    career_plan = json.loads(career_plan_json_text)
    
    # Print the parsed career plan to verify
    print("Successfully parsed career plan:")
    pprint(career_plan)
    
except Exception as e:
    raise ValueError(f"Failed to parse AI response as JSON: {e}")

# 8. Find Skill Development Resources with Google Search Grounding & Few-Shot Prompting

In [ ]:
# Use Gemini's built-in Google Search tool
model_id = "gemini-2.0-flash"
google_search_tool = Tool(google_search=GoogleSearch())

# Provide few-shot examples for search results
search_result_examples ="""
[{
  "skill": "Product Strategy (developing product vision and roadmaps)",
  "resources": [
    {
      "title": "Product Strategy: Develop and Manage Successful Products",
      "provider": "Kellogg School of Management",
      "description": "Designed to train business stakeholders. The course content will enable executives to prepare wireframes, decide pricing models for SaaS, and guide product launch and subsequent revisions for growth strategies.",
      "link": "https://www.kellogg.northwestern.edu/executive-education/individual-programs/online-programs/prodstrat.aspx"
    },
    ...,
    {
      "title": "Mastering Product Management: Building Your Strategy",
      "provider": "Stanford Online",
      "description": "Provides a structured playbook for developing product strategies aligned with company goals. It covers essential techniques such as creating roadmaps, managing product teams, and driving product growth.",
      "link": "https://online.stanford.edu/courses/xprod210-mastering-product-management-building-your-strategy"
    }
  ]
}],
[{
  "skill": "Market Research (analyzing market trends and competitive landscape)",
  "resources": [
    {
      "title": "Market Research Specialization",
      "provider": "University of California, Davis on Coursera",
      "description": "Offers a comprehensive approach to market research, suitable for various experience levels.",
      "link": "https://www.coursera.org/specializations/market-research"
    },
    ...,
    {
      "title": "Market Research and Consumer Behavior",
      "provider": "IE Business School",
      "description": "Focuses on understanding consumer behavior and how it impacts market research.",
      "link": "https://www.ie.edu/business-school/programs/masters/master-in-market-research-consumer-behavior/"
    }
  ]
}],
...,
[{
  "skill": "User Research (understanding user needs and pain points)",
  "resources": [
    {
      "title": "Google UX Design Certificate",
      "provider": "Coursera",
      "description": "A beginner-friendly course that covers the basics of user experience design, including user research, ideation, wireframing, and prototyping.",
      "link": "https://www.coursera.org/professional-certificates/google-ux-design"
    },
    ...,
    {
      "title": "User Research – Methods and Best Practices",
      "provider": "The Interaction Design Foundation",
      "description": "Provides in-depth training on user research methods, focusing on practical applications for product and UX design.",
      "link": "https://www.interaction-design.org/courses/user-research-methods-and-best-practices?srsltid=AfmBOorDt8DQBhRiUZSEdjFUSDdxuPgMXezdL7G91IpBCiZg5XAIfvhr"
    }
  ]
}]
"""

# Create a main query for the search
main_search_query = f"""Return this in JSON format matching my examples exactly. For each of the following skills, list 2-3 reputable online courses, certifications or other learning resources:

Skills: {', '.join(career_plan['missing_skills'])}

The output should be a JSON array where each element follows this structure:
[{{
  "skill": "[Skill Name]",
  "resources": [
    {{
      "title": "[Resource Title]",
      "provider": "[Provider Name]",
      "description": "[Brief description of the resource]",
      "link": "[Resource URL]"
    }},
    ...,
    {{
      "title": "[Resource Title]",
      "provider": "[Provider Name]",
      "description": "[Brief description of the resource]",
      "link": "[Resource URL]"
    }},
  ]
}}]"""

# Combine few-shot examples with main search query to create few-shot prompt
few_shot_prompt = search_result_examples + "\n" + main_search_query

# Send the few-shot prompt to Gemini and have it use the Google Search tool
search_response = client.models.generate_content(
    model=model_id,
    contents=few_shot_prompt,
    config=GenerateContentConfig(
        tools=[google_search_tool],
        response_modalities=["TEXT"]
    )
)

# 9. Parse and Format Skill Development Resources from Search Results

In [ ]:
# Get the raw text from the response
search_response_text = search_response.candidates[0].content.parts[0].text

# Extract JSON from markdown code blocks if present
if "```json" in search_response_text:
    search_json_text = search_response_text.split("```json")[1].split("```")[0].strip()
elif "```" in search_response_text:
    search_json_text = search_response_text.split("```")[1].split("```")[0].strip()
else:
    search_json_text = search_response_text

# Parse the JSON
try:
    search_resources_data = json.loads(search_json_text)
    
    # Format the resources as markdown
    search_resources_markdown = ""
    
    for item in search_resources_data:
        skill = item.get("skill", "")
        search_resources_markdown += f"**{skill}**\n"
        
        for resource in item.get("resources", []):
            title = resource.get("title", "")
            provider = resource.get("provider", "")
            description = resource.get("description", "")
            link = resource.get("link", "")
            
            search_resources_markdown += f"* **{title} ({provider}):** {description} [Link]({link})\n"
        
        search_resources_markdown += "\n"
    
except Exception as e:
    search_resources_markdown = f"Error processing search resources: {str(e)}\n\nRaw response: {search_response_text}"


# 10. Display Roadmap + Grounded Search Results

In [ ]:
from IPython.display import display, Markdown, HTML

# Create the complete markdown output from the career plan and the 
full_markdown_output = f"""
<h1 align="center">Career Transition Plan</h1>

<h3 align="center">Target Role: {target_role}</h3>
<h3 align="center">Sector: {target_sector}</h3>

### 💱 Transferable Skills (with evidence):
{chr(10).join([f"- **{skill['name']}** - {skill['evidence']}" for skill in career_plan['transferable_skills']])}

### 🏋 Skills to Learn:
{chr(10).join([f"- {skill}" for skill in career_plan['missing_skills']])}

### 📜 Recommended Certifications:
{chr(10).join([f"- {cert['title']} from {cert['provider']}" for cert in career_plan['recommended_certifications']])}

### 👍 Recommended Learning Resources:
{chr(10).join([f"- {resource['title']} ({resource['provider']})" for resource in career_plan['learning_resources']])}

### 🚶‍➡️ Career Pathway:
{" → ".join(career_plan['intermediate_roles'])}

### 🗺️ Roadmap Summary:
{career_plan['roadmap_summary']}

### 🔍 Skill Development Opportunities (via Google Search Grounding)
{search_resources_markdown}
"""
display(Markdown(full_markdown_output))

# Conclusion & Reflections

In this notebook, we explored how Generative AI can:
- Analyze documents (Resume PDF)
- Compare a user's experience to a target role and identify transferable and missing skills
- Generate personalized career roadmap outputs in structured JSON
- Use search grounding to recommend up-to-date learning resources using few-shot prompting to guide results

This application can support students and job seekers in navigating career pivots with clarity and confidence. Future expansions could include resume editing, interview prep, or integration with job boards and online learning platforms.